# AI SQL Querier

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-OpenAI-API-KEY" data-toc-modified-id="1.-OpenAI-API-KEY-1">1. OpenAI API KEY</a></span></li><li><span><a href="#2.-SQL-Connection" data-toc-modified-id="2.-SQL-Connection-2">2. SQL Connection</a></span></li><li><span><a href="#3.-Load-SQL-DB-in-LangChain" data-toc-modified-id="3.-Load-SQL-DB-in-LangChain-3">3. Load SQL DB in LangChain</a></span></li><li><span><a href="#4.-Input-LLM" data-toc-modified-id="4.-Input-LLM-4">4. Input LLM</a></span></li><li><span><a href="#5.-SQL-Chain" data-toc-modified-id="5.-SQL-Chain-5">5. SQL Chain</a></span></li><li><span><a href="#6.-Executing-generated-SQL-query" data-toc-modified-id="6.-Executing-generated-SQL-query-6">6. Executing generated SQL query</a></span></li><li><span><a href="#7.-Cleaning-response" data-toc-modified-id="7.-Cleaning-response-7">7. Cleaning response</a></span></li><li><span><a href="#8.-Output-LLM" data-toc-modified-id="8.-Output-LLM-8">8. Output LLM</a></span></li><li><span><a href="#9.-Code-Summary" data-toc-modified-id="9.-Code-Summary-9">9. Code Summary</a></span></li></ul></div>

## 1. OpenAI API KEY

To carry out this project, we will need an API KEY from OpenAI to use the GPT-4 Turbo model. This API KEY can be obtained at https://platform.openai.com/api-keys. It is only displayed once, so it must be saved at the moment it is obtained. Of course, we will need to create an account to get it.

We store the API KEY in a `.env` file to load it with the dotenv library and use it as an environment variable. This file is added to the `.gitignore` to ensure that it cannot be seen if we upload the code to GitHub, for example.

In [1]:
# removing deprecation warnings

import warnings
warnings.filterwarnings("ignore")

In [2]:
# import API KEY

import os                           # operating system library
from dotenv import load_dotenv      # load environment variables  


load_dotenv()


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## 2. SQL Connection

We make the SQL connection by creating the URI, with the SnowFlake credentials. These details will change depending on the database used and the user credentials.

In [3]:
from sqlalchemy import create_engine

In [4]:
# server and user credentials 

snowflake_account = os.getenv("snowflake_account")

username = os.getenv("username")

password = os.getenv("password")

database = os.getenv("database")

schema = os.getenv("schema")

warehouse = os.getenv("warehouse")

role = os.getenv("role")

In [5]:
URI = f"snowflake://{username}:{password}@{snowflake_account}/{database}/{schema}?warehouse={warehouse}&role={role}"

In [6]:
# database connection

cursor = create_engine(URI).connect()

In [7]:
# show all database tables

tables = cursor.execute("show tables;").fetchall()

tables = [e[1] for e in tables]

tables

['`ACTOR`',
 '`ADDRESS`',
 '`CATEGORY`',
 '`CITY`',
 '`COUNTRY`',
 '`CUSTOMER`',
 '`FILM_ACTOR`',
 '`FILM_CATEGORY`',
 '`FILM_TEXT`',
 '`FILM`',
 '`INVENTORY`',
 '`LANGUAGE`',
 '`PAYMENT`',
 '`RENTAL`',
 '`STAFF`',
 '`STORE`']

## 3. Load SQL DB in LangChain

We create the SQL chain for feeding the LLM later.

In [8]:
from langchain import SQLDatabase

In [9]:
db = SQLDatabase.from_uri(URI,
                          sample_rows_in_table_info=1, 
                          include_tables=tables)


In [10]:
# tables information

print(db.table_info)


CREATE TABLE "`ACTOR`" (
	"`ACTOR_ID`" DECIMAL(38, 0) NOT NULL, 
	"`FIRST_NAME`" VARCHAR(45) NOT NULL, 
	"`LAST_NAME`" VARCHAR(45) NOT NULL, 
	"`LAST_UPDATE`" TIMESTAMP_NTZ NOT NULL, 
	CONSTRAINT "SYS_CONSTRAINT_a33a12cb-9ed4-4179-ad31-82eddacc6b7b" PRIMARY KEY ("`ACTOR_ID`")
)

/*
1 rows from `ACTOR` table:
`ACTOR_ID`	`FIRST_NAME`	`LAST_NAME`	`LAST_UPDATE`
1	PENELOPE	GUINESS	2006-02-15 03:34:33
*/


CREATE TABLE "`ADDRESS`" (
	"`ADDRESS_ID`" DECIMAL(38, 0) NOT NULL, 
	"`ADDRESS`" VARCHAR(16777216) NOT NULL, 
	"`ADDRESS2`" VARCHAR(16777216), 
	"`DISTRICT`" VARCHAR(16777216) NOT NULL, 
	"`CITY_ID`" DECIMAL(38, 0) NOT NULL, 
	"`POSTAL_CODE`" VARCHAR(10), 
	"`PHONE`" VARCHAR(16777216) NOT NULL, 
	"`LOCATION`" VARCHAR(16777216) NOT NULL, 
	"`LAST_UPDATE`" TIMESTAMP_NTZ NOT NULL
)

/*
1 rows from `ADDRESS` table:
`ADDRESS_ID`	`ADDRESS`	`ADDRESS2`	`DISTRICT`	`CITY_ID`	`POSTAL_CODE`	`PHONE`	`LOCATION`	`LAST_UPDATE`
1	47 MySakila Drive	None	Alberta	300			A	2014-09-25 20:30:27
*/


CREATE TABL

## 4. Input LLM

We use an OpenAI model as input LLm. A temperature = 0 makes the model's responses more deterministic and repetitive, tending to select the most likely next word given the previous context, which is better for SQL queries generation.

In [11]:
from langchain_openai import OpenAI

In [12]:
input_model = OpenAI(temperature=0)

In [13]:
# testing model

input_model.invoke("What is SQL?")

'\n\nSQL (Structured Query Language) is a programming language used for managing and manipulating data stored in relational databases. It is used to create, retrieve, update, and delete data from databases, as well as to define and modify the structure of databases. SQL is a standard language that is used by many database management systems, making it a widely used and important tool for data management and analysis.'

## 5. SQL Chain

We use our database and the LLM from OpenAI for SQL query generation.

In [14]:
from langchain.chains import create_sql_query_chain

In [15]:
database_chain = create_sql_query_chain(input_model, db)

In [16]:
# chain description

database_chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for (k, v) in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'dialect': 'snowflake', 'top_k': '5'}, template='Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a the few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attentio

In [17]:
# our question

prompt = "Which actors have the first name ‘SCARLETT’? Add `` to table name"

In [18]:
# generated query

sql_query = database_chain.invoke({"question": prompt})

In [19]:
#we can visualize what sql query is generated by the LLM

print(sql_query)

SELECT `FIRST_NAME`, `LAST_NAME` FROM `ACTOR` WHERE `FIRST_NAME` = 'SCARLETT' LIMIT 5


## 6. Executing generated SQL query

In [20]:
# SQL output

response = cursor.execute(sql_query).fetchall()

response

[('SCARLETT', 'DAMON'), ('SCARLETT', 'BENING')]

## 7. Cleaning response

We clean the SQL output so that the model understands better and returns a more concise response parsing the output to markdown.

In [21]:
import pandas as pd

In [22]:
context = pd.DataFrame(response).to_markdown()

context

'|    | `FIRST_NAME`   | `LAST_NAME`   |\n|---:|:---------------|:--------------|\n|  0 | SCARLETT       | DAMON         |\n|  1 | SCARLETT       | BENING        |'

## 8. Output LLM

With this model, we generate the final response.

In [23]:
from langchain_openai.chat_models import ChatOpenAI   

output_model = ChatOpenAI(model="gpt-4-turbo")

In [24]:
output_model.invoke(f"""Given the next context, answer the cuestion: 
                    
                    context: {context}, 
                    
                    question: {prompt}
                    
                    """).content

"The actors with the first name 'SCARLETT' from the table `context` are:\n\n- SCARLETT DAMON\n- SCARLETT BENING"

## 9. Code Summary

In [25]:
%%time

import warnings
warnings.filterwarnings("ignore")

from sqlalchemy import create_engine

from langchain import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import OpenAI
from langchain_openai.chat_models import ChatOpenAI   

import pandas as pd

import os                           
from dotenv import load_dotenv        




load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


snowflake_account = os.getenv("snowflake_account")

username = os.getenv("username")

password = os.getenv("password")

database = os.getenv("database")

schema = os.getenv("schema")

warehouse = os.getenv("warehouse")

role = os.getenv("role")


URI = f"snowflake://{username}:{password}@{snowflake_account}/{database}/{schema}?warehouse={warehouse}&role={role}"
    
    
cursor = create_engine(URI).connect()


tables = cursor.execute("show tables;").fetchall()

tables = [e[1] for e in tables]


db = SQLDatabase.from_uri(URI,
                          sample_rows_in_table_info=1, 
                          include_tables=tables)

input_model = OpenAI(temperature=0)


database_chain = create_sql_query_chain(input_model, db)


prompt = "Which actors have the first name ‘SCARLETT’? Add `` to table name"


sql_query = database_chain.invoke({"question": prompt})


response = cursor.execute(sql_query).fetchall()


context = pd.DataFrame(response).to_markdown()


output_model = ChatOpenAI(model="gpt-4-turbo")



final_prompt = f"""Given the next context, answer the cuestion: 
                    
                   context: {context}, 
                    
                   question: {prompt}
                    
                   """



output_model.invoke(final_prompt).content

CPU times: user 822 ms, sys: 81 ms, total: 903 ms
Wall time: 11.7 s


"The actors with the first name 'SCARLETT' are:\n\n- SCARLETT DAMON\n- SCARLETT BENING"